In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm

from scipy.stats import norm
from tabulate import tabulate

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize']=[24.0,8.0]
matplotlib.rcParams['font.size']=10
matplotlib.rcParams['lines.linewidth']=2.0

import cufflinks as cf
cf.set_config_file(offline=True,dimensions=((1000,600)))

import plotly.express as px
px.defaults.template, px.defaults.width, px.defaults.height = "plotly_white", 1000, 600


In [7]:
from helper import plot_asset_path, plot_probability, plot_binomial_tree

ModuleNotFoundError: No module named 'grp'

In [ ]:
#plot settings

plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize']=[12.0,8.0]
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['lines.linewidth'] = 2.0

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('Nasdaq.csv',index_col=0,parse_dates=True)

In [ ]:
df.head()
df.index

In [ ]:
#!pip install ipyplot

In [ ]:
#specify date first

df['Adj Close'].iplot()



In [ ]:
df['Adj Close'][-300:].iplot()

In [ ]:
df.index

In [ ]:
df=pd.read_csv('Nasdaq.csv',index_col=0,parse_dates=True,dayfirst=True)['2011':'2022']

#dayfirst parameter to fix date format issue in pandas

#calculating percentage change

df['Change']=np.log(df['Adj Close']).diff().fillna(0)*100

In [ ]:
df['Adj Close'].iplot()

In [ ]:
df['Adj Close'][-300:].iplot()

In [ ]:
df1=df.copy() #copy does not affect the orginal dataframe

In [ ]:
df1.reset_index(inplace=True)
df1.tail()


In [ ]:
df1['Year']=df1['Date'].dt.year
df1['Month']=df1['Date'].dt.month


In [ ]:
df1.tail()

In [ ]:
df1.groupby('Year')['Change'].idxmax()

In [ ]:
#check the yearwise max values

df1.loc[df1.groupby('Year')['Change'].idxmax()]

In [ ]:
df1.loc[df1.groupby('Year')['Change'].idxmin()]

In [ ]:
#Assign a new dataframe with pivoted values

newdf=pd.pivot_table(df1,
                    index='Month',
                    columns='Year',
                    values='Change',
                    aggfunc=np.sum
                    )

newdf

In [ ]:
newdf.describe()

In [ ]:
#Box Plot Analysis

newdf.iplot(kind='box',
           title='Nasdaq Return Analyssi',
           yTitle='% chng',
           legend=False,
           boxpoints='outliers')

# Modeling Returns

In [ ]:
nasdaq=pd.read_csv('Nasdaq.csv',index_col=0,parse_dates=True,dayfirst=True)

In [ ]:
nasdaq['Return']=nasdaq['Adj Close'].pct_change().fillna(0)

In [ ]:
nasdaq.head()

In [ ]:
#Calculate mean and std deviation

In [ ]:
mu=np.mean(nasdaq['Return'])
mu

In [ ]:
sigma=np.std(nasdaq['Return'])
sigma

In [ ]:
#Calculate scaled return

nasdaq['Scaled_Return']=nasdaq['Return'].apply(lambda x: (x-mu)/sigma)
nasdaq.head()

In [ ]:
nasdaq['Scaled_Return'].max(), nasdaq['Scaled_Return'].min()

In [ ]:
x=np.linspace(nasdaq['Scaled_Return'].min(), nasdaq['Scaled_Return'].max(),200)

In [ ]:
y=(1/np.sqrt(2*np.pi)*np.exp(-0.5*x**2))

In [ ]:
plt.hist(nasdaq['Scaled_Return'],bins=200,density=True,label='Empirical',color='red',alpha=0.75)

#superimpose normal distribution curve

plt.plot(x,y,color='green',label='Normal',alpha=1) #alpha controls the depth of the color
plt.xlim(-5,5)
plt.ylim(0,0.8)
plt.title('Empirical vs Normal')
plt.show()

# Binomial Tree

In [5]:
#import Math functions from numpy
from numpy import *

In [6]:
#import plotting functions from helper

from helper import plot_asset_path, plot_probability, plot_binomial_tree

ModuleNotFoundError: No module named 'grp'

In [ ]:
plot_asset_path()

In [ ]:
# Path Probability

plot_probability()

In [ ]:
def binomial_option(spot, strike, rate, sigma, time, steps, output=0):
    
    """
    binomial_option(spot, strike, rate, sigma, time, steps, output=0)
    
    Function for building binomial option tree for european call option payoff. 
    
    Parameters
    ----------
    spot        int or float   - spot price
    strike      int or float   - strike price 
    rate        float          - interest rate
    sigma       float          - volatility
    time        int or float   - expiration time
    steps       int            - number of time steps
    output      int            - [0: price, 1: payoff, 2: option value, 3: option delta]
    
    Returns
    ----------
    out : ndarray
    An array object of price, payoff, option value and delta as specified by the output parameter

    """
    
    # define parameters
    ts = time/steps                                 # ts is time steps, dt
    u = 1 + sigma*sqrt(ts)                          # u is up factor
    v = 1 - sigma*sqrt(ts)                          # v is down factor
    p = 0.5 + rate*sqrt(ts)/(2*sigma)               # p here is risk neutral probability (p') - for ease of use
    df = 1/(1+rate*ts)                              # df is discount factor

    # initialize arrays
    px = zeros((steps+1, steps+1))                  # price path
    cp = zeros((steps+1, steps+1))                  # call intrinsic payoff
    V = zeros((steps+1, steps+1))                   # option value
    d = zeros((steps+1, steps+1))                   # delta value
    
    # binomial loop
    for j in range(steps+1):
        for i in range(j+1):
            px[i,j] = spot * power(v,i) * power(u,j-i)
            cp[i,j] = maximum(px[i,j] - strike,0)
            
    for j in range(steps+1, 0, -1):
        for i in range(j):
            if (j == steps+1):
                V[i,j-1] = cp[i,j-1]                # terminal payoff
                d[i,j-1] = 0                        # terminal delta
            else:
                V[i,j-1] = df*(p*V[i,j]+(1-p)*V[i+1,j])
                d[i,j-1] = (V[i,j]-V[i+1,j])/(px[i,j]-px[i+1,j])
                    
    results = around(px,2), around(cp,2), around(V,2), around(d,4)
    
    return results[output]

In [ ]:
px = binomial_option(100,100,0.05,0.2,1,4,0)
px

In [ ]:
option_price=binomial_option(100,100,0.05,0.2,1,4,2)
option_price

In [ ]:
option_price[0][0]

In [ ]:
option_delta=binomial_option(100,100,0.05,0.2,1,4,3)
option_delta

In [ ]:
# Visualizing the Binomial Tree

plot_binomial_tree(px[0,0], px, option_price, option_delta)

In [ ]:
opt_price= binomial_option(100,100,0.05,0.3,1,4,2)
opt_price[0][0]

In [ ]:
# Calculating value of Option for a range of volatilities and plotting

In [ ]:
vol_range = linspace(0.02,0.5,100)

def opt_price(vol):
    
    opt_price=[]
    
    for vol in vol_range:
        px= binomial_option(100,100,0.05,vol,1,4,2)
        opt_price.append(px[0][0])
        
    return pd.DataFrame({'Opt_Price' : opt_price,
                         'Volatility' : vol_range})
        
        
    

In [ ]:
option_price = opt_price(vol_range)
option_price.head()

In [ ]:
plt.plot(option_price['Volatility'],option_price['Opt_Price'],alpha=0.9,color='b')

In [ ]:
# Calculating value of Option as the number of time steps (NTS) increases

In [ ]:
NTS = range(4,20)
vol = 0.20

def opt_price(NTS):
    
    opt_price=[]
    
    for t in NTS:
        px= binomial_option(100,100,0.05,vol,1,t,2)
        opt_price.append(px[0][0])
        
    return pd.DataFrame({'Opt_Price' : opt_price,
                         'NTS' : NTS})

In [ ]:
Option_Price_NTS=opt_price(NTS)
Option_Price_NTS.head()

In [ ]:
plt.plot(Option_Price_NTS['NTS'],Option_Price_NTS['Opt_Price'],color='b',alpha=1)